# Dataset creation

> CLI commands for creating different types of datasets from remote sensing data

In [ ]:
#| default_exp scripts.data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from fastcore.script import *
import datetime
import geopandas as gpd
import rasterio as rio
from pathlib import Path
import os

from geo2ml.data.tabular import *
from geo2ml.data.tiling import Tiler
from geo2ml.data.cv import *

## Overview

`geo2ml` provides the following commands for creating datasets from geospatial raster and vector data.

* `geo2ml_sample_points`
* `geo2ml_sample_polygons`
* `geo2ml_create_raster_dataset`
* `geo2ml_create_yolo_dataset`
* `geo2ml_create_coco_dataset`

These commands can be either used from CLI using `geo2ml_` -prefixed commands, or used in python scripts or notebooks like 

```python
from geo2ml.scripts.data import sample_points

sampling_locations = Path(<path_to_locations>)
input_raster = Path(<path_to_raster>)
target_column = 'column'
outpath = Path(<path_to_save_files>)

sample_points(sampling_locations, input_raster, target_column, outpath)
```

## Tabular datasets

Both of these commands create a dataset by sampling point or polygon values provided in `sampling_locations` from `input_raster` and save the resulting table as a csv and geojson or shapfile to `outpath`.

In [ ]:
#| export

@call_parse
def sample_points(sampling_locations:Path, # Path to the geojson/shapefile containing the sampling locations as points
                  input_raster:Path, # Path to the raster used for sampling
                  target_column:str, # Column of `sampling_locations` used as the target
                  outpath:Path, # Path to save the output files. Is created if doesn't exist
                  gpkg_layer:str=None, # If `sampling_locations` is .gpkg, specify the layer used. Ignored otherwise.
                  save_as_shp:bool=False, # Save results as shapefiles? If False, saves as geojson
                  rename_target:str=None, # If provided, target column is renamed to this
                  band_names:Path=None, # Path to a file providing bands to use as rows
                  dropna_value:int=None, # Drop all rows with all values equal to this value
                  out_prefix:str=""): # Prefix for outputs
    "Sample pixel values from `input_raster` using `sampling_locations`" 

    if band_names:
        with open(band_names) as f: 
            bandnames = [l.strip() for l in f.readlines()]
    
    gdf = sample_raster_with_points(sampling_locations, input_raster, target_column, band_names=band_names)

    df = gdf.drop(columns=['geometry'])

    if dropna_value:
        df = df.loc[~(df[bands] == dropna_value).all(axis=1)]
    df = df.dropna().reset_index(drop=True)

    shp_stem = sampling_locations.stem
    raster_stem = input_raster.stem

    outpath.mkdir(parents=True, exist_ok=True)
    out_stem = Path(f'{raster_stem}__{shp_stem}__{target_column}')

    if out_prefix != "":
        out_stem = Path(f'{out_prefix}__{out_stem}')

    suffix = '.shp' if save_as_shp else '.geojson'

    gdf.to_file(outpath/out_stem.with_suffix(suffix))
    df.to_csv(outpath/out_stem.with_suffix('.csv'), index=False)

In [ ]:
#| export

@call_parse
def sample_polygons(sampling_locations:Path, # Path to the geojson/shapefile containing the sampling locations as polygons
                    input_raster:Path, # Path to the raster used for sampling
                    target_column:str, # Column of `sampling_locations` used for sampling
                    outpath:Path, # Path to save the output files. Is created if doesn't exist
                    min:bool, # Compute minimum
                    max:bool, # Compute maximum
                    mean:bool, # Compute mean
                    count:bool, # Compute count
                    sum:bool, # Compute sum
                    std:bool, # Compute standard deviation
                    median:bool, # Compute median
                    categorical:bool=False, # Are bands categorical data?
                    gpkg_layer:str=None, # If `sampling_locations` is .gpkg, specify the layer used. Ignored otherwise.
                    save_as_shp:bool=False, # Save results as shapefiles? If False, saves as geojson
                    rename_target:str=None, # If provided, target column is renamed to this
                    band_names:Path=None, # Path to a file providing bands to use as rows
                    dropna_value:int=None, # Drop all rows with all values equal to this value
                    out_prefix:str=""): # Prefix for outputs
    "Sample pixel values from `input_raster` using `sampling_locations`. " 

    stats = []
    if min: stats.append('min')
    if max: stats.append('max')
    if mean: stats.append('mean')
    if count: stats.append('count')
    if sum: stats.append('sum')
    if std: stats.append('std')
    if median: stats.append('median')

    if band_names:
        with open(band_names) as f: 
            bandnames = [l.strip() for l in f.readlines()]
    
    gdf = sample_raster_with_polygons(sampling_locations, input_raster, target_column, band_names=band_names,
                                      stats=stats, categorical=categorical)

    df = gdf.drop(columns=['geometry'])

    if dropna_value:
        df = df.loc[~(df[bands] == dropna_value).all(axis=1)]
    df = df.dropna().reset_index(drop=True)

    shp_stem = sampling_locations.stem
    raster_stem = input_raster.stem

    outpath.mkdir(parents=True, exist_ok=True)
    out_stem = Path(f'{raster_stem}__{shp_stem}__{target_column}')

    if out_prefix != "":
        out_stem = Path(f'{out_prefix}__{out_stem}')

    suffix = '.shp' if save_as_shp else '.geojson'

    gdf.to_file(outpath/out_stem.with_suffix(suffix))
    df.to_csv(outpath/out_stem.with_suffix('.csv'), index=False)

## Computer vision dataset creation

In [ ]:
#| export

@call_parse
def create_raster_dataset(
    raster_path:Path, # Path to a raster that is the base for the dataset
    mask_path:Path, # Path to corresponding mask raster or polygon layer. Must have the same extent and resolution as the raster in `raster_path`
    outpath:Path, # Where to save the results
    save_grid:bool=False, # Whether to save the tiling grid
    target_column:str=None, # If mask_path contains vector data, identifier of the column containing the class information
    gpkg_layer:str=None, # If `polygon_path` is a geopackage, specify the layer used. Ignored otherwise.
    gridsize_x:int=256, # Size of tiles in x-axis in pixels
    gridsize_y:int=256, # Size of tiles in y-axis in pixels
    overlap_x:int=0, # Overlap of tiles in x-axis in pixels
    overlap_y:int=0, # Overlap of tiles in y-axis in pixels
):
    "Create a semantic segmentation dataset from a `raster_path` and corresponding mask `mask_path`. Raster image patches are saved to `outpath/raster_tiles` and mask patches to `outpath/mask_tiles`"
    tiler = Tiler(outpath, gridsize_x=gridsize_x, gridsize_y=gridsize_y, overlap=(overlap_x, overlap_y))
    tiler.tile_raster(raster_path)
    
    polygon_extensions = ['.shp', '.geojson', '.gpkg']
    raster_extensions = ['.tif']

    if mask_path.suffix in raster_extensions:
        tiler.raster_path = outpath/'mask_images'
        tiler.tile_raster(mask_path)
    elif mask_path.suffix in polygon_extensions:
        if not target_column:
            raise Exception(
                "If mask_path contains polygon data, target_column must be provided"
            )
        tiler.tile_and_rasterize_vector(raster_path, mask_path, column=target_column, gpkg_layer=gpkg_layer)
        os.rename(tiler.rasterized_vector_path, outpath/'mask_images')
    if save_grid: tiler.grid.to_file(outpath/'grid.geojson')

In [ ]:
#| export

@call_parse
def create_coco_dataset(
    raster_path:Path, # Path to a raster that is the base for the dataset
    polygon_path:Path, # Path to annotated polygons
    target_column:str, # Which column contains class information
    outpath:Path, # Where to save the resulting files
    dataset_name:str, # Name of the dataset
    gpkg_layer:str=None, # If `polygon_path` is a geopackage, specify the layer used. Ignored otherwise.
    min_area_pct:float=0.0, # How small polygons keep after tiling?
    save_grid:bool=False, # Should tiling grid be saved
    gridsize_x:int=320, # Size of tiles in x-axis in pixels
    gridsize_y:int=320, # Size of tiles in y-axis in pixels
    overlap_x:int=0, # Overlap of tiles in x-axis in pixels
    overlap_y:int=0, # Overlap of tiles in y-axis in pixels
    ann_format:str='box', # Annotation format, either box, polygon or rotated box
    min_bbox_area:int=0 # Minimum bounding gox area in pixels. Smaller objects than this are discarded
):
    "Create a COCO-format dataset from `raster` and `polygon` shapefile"
    tiler = Tiler(outpath, gridsize_x=gridsize_x, gridsize_y=gridsize_y, overlap=(overlap_x, overlap_y))
    tiler.tile_raster(raster_path)
    tiler.tile_vector(polygon_path)

    cats = gpd.read_file(polygon_path)[target_column].unique()

    coco_cats = [{'supercategory':'object', 'id':i+1, 'name':c} for i, c in enumerate(cats)]
    coco_info = {'description': dataset_name,
                 'version': 0.1,
                 'year': datetime.date.today().year,
                 'contributor': "Your name",
                 'date_created': datetime.date.today().strftime("%Y/%m/%d")
    }

    coco_licenses = {}
    shp_to_coco(outpath/'images', outpath/'vectors', outpath, label_col=target_column, 
                dataset_name=dataset_name, coco_info=coco_info, coco_categories=coco_cats,
                min_bbox_area=min_bbox_area, rotated_bbox=ann_format=='rotated box')
    if save_grid: tiler.grid.to_file(outpath/'grid.geojson')

In [ ]:
#| export

@call_parse
def create_yolo_dataset(
    raster_path:Path, # Path to a raster that is the base for the dataset
    polygon_path:Path, # Path to annotated polygons
    target_column:str, # Which column contains class information
    outpath:Path, # Where to save the resulting files?
    dataset_name:str=None, # Optional name of the dataset
    gpkg_layer:str=None, # If `polygon_path` is a geopackage, specify the layer used. Ignored otherwise.
    min_area_pct:float=0.0, # How small polygons keep after tiling?
    save_grid:bool=False, # Should tiling grid be saved?
    gridsize_x:int=320, # Size of tiles in x-axis, pixels
    gridsize_y:int=320, # Size fo tiles in y-axis, pixels
    overlap_x:int=0, # Overlap of tiles in x-axis
    overlap_y:int=0, # Overlap of tiles in y-axis
    ann_format:str='box', # Annotation format, either box, polygon or rotated box
    min_bbox_area:int=0 # Minimum bounding box area in pixels. Smaller objects than this are discarded
):
    "Create a YOLO-format dataset from `raster` and `polygon` shapefile"
    tiler = Tiler(outpath, gridsize_x=gridsize_x, gridsize_y=gridsize_y, overlap=(overlap_x, overlap_y))
    tiler.tile_raster(raster_path)
    tiler.tile_vector(polygon_path)
    cats = gpd.read_file(polygon_path)[target_column].unique()
    shp_to_yolo(outpath/'images', outpath/'vectors', outpath, label_col=target_column,
                names=cats, dataset_name=dataset_name, ann_format=ann_format, min_bbox_area=0)
    if save_grid: tiler.grid.to_file(outpath/'grid.geojson')